# ⛓ Chain 

In [ ]:
#| default_exp chains

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import asyncio, web3, os
from functools import wraps, reduce
from typing import List, TypeVar, Callable, Optional, Tuple
from fastcore.utils import patch
from web3 import AsyncWeb3, AsyncHTTPProvider, Account
from web3.eth.async_eth import AsyncContract
from sugar.config import ChainSettings, make_op_chain_settings, make_base_chain_settings
from sugar.helpers import normalize_address, MAX_UINT256, float_to_uint256, apply_slippage, get_future_timestamp
from sugar.abi import sugar, price_oracle, router
from sugar.token import Token
from sugar.pool import LiquidityPool
from sugar.price import Price
from sugar.deposit import Deposit
from sugar.helpers import ADDRESS_ZERO, chunk

## Chain implementation 

In [ ]:
#| export

T = TypeVar('T')

def require_context(f: Callable[..., T]) -> Callable[..., T]:
    @wraps(f)
    async def wrapper(self: 'Chain', *args, **kwargs) -> T:
        if not self._in_context: raise RuntimeError("Chain methods can only be accessed within 'async with' block")
        return await f(self, *args, **kwargs)
    return wrapper

class Chain:
    account: Optional[Account]
    web3: AsyncWeb3
    sugar: AsyncContract
    router: AsyncContract

    def __init__(self, settings: ChainSettings):
        self.settings, self._in_context = settings, False

    @property
    def account(self) -> Account: return self.web3.eth.account.from_key(os.getenv("SUGAR_PK"))

    async def __aenter__(self):
        """Async context manager entry"""
        self._in_context = True
        self.web3 = AsyncWeb3(AsyncHTTPProvider(self.settings.rpc_uri))
        self.sugar = self.web3.eth.contract(address=self.settings.sugar_contract_addr, abi=sugar)
        self.prices = self.web3.eth.contract(address=self.settings.price_oracle_contract_addr, abi=price_oracle)
        self.router = self.web3.eth.contract(address=self.settings.router_contract_addr, abi=router)
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Async context manager exit"""
        self._in_context = False
        await self.web3.provider.disconnect()
        return None

### Get tokens

In [ ]:
#| export

@patch
@require_context
async def get_all_tokens(self: Chain, listed_only: bool = True) -> List[Token]:
    tokens = list(map(lambda t: Token.from_tuple(t), await self.sugar.functions.tokens(self.settings.pagination_limit, 0, ADDRESS_ZERO, []).call()))
    return list(filter(lambda t: t.listed, tokens)) if listed_only else tokens
   

### Get pools

In [ ]:
#| export

@patch
@require_context
async def get_pools(self: Chain) -> List[LiquidityPool]:
    pools, offset, limit = [], 0, self.settings.pool_page_size
    tokens = await self.get_all_tokens()
    prices = await self.get_prices(tokens)
    tokens = {t.token_address: t for t in tokens}
    prices = {price.token.token_address: price for price in prices}

    while True:
        pools_batch = await self.sugar.functions.all(limit, offset).call()
        pools += pools_batch
        if len(pools_batch) < limit: break
        else: offset += limit

    return list(filter(lambda p: p is not None, map(lambda p: LiquidityPool.from_tuple(p, tokens), pools)))

### Get prices

In [ ]:
#| export

# @cache_in_seconds(ORACLE_PRICES_CACHE_MINUTES * 60)
@patch
async def _get_prices(self: Chain, tokens: Tuple[Token]):
    prices = await self.prices.functions.getManyRatesWithCustomConnectors(
        list(map(lambda t: t.token_address, tokens)),
        self.settings.stable_token_addr,
        False, # use wrappers
        self.settings.connector_tokens_addrs,
        10 # threshold_filer
    ).call()
    # 6 decimals for USDC
    return [Price(token=tokens[cnt], price=price / 10**6) for cnt, price in enumerate(prices)]

@patch
@require_context
async def get_prices(self: Chain, tokens: List[Token]) -> List[Price]:
    """Get prices for tokens in target stable token"""
    # filter out stable token from tokens list so getManyRatesWithCustomConnectors so does not freak out
    tokens_without_stable = list(filter(lambda t: t.token_address != self.settings.stable_token_addr, tokens))
    stable = next(filter(lambda t: t.token_address == self.settings.stable_token_addr, tokens), None)

    batches = await asyncio.gather(
        *map(
            # XX: lists are not cacheable, convert them to tuples so lru cache is happy
            lambda ts: self._get_prices(tuple(ts)),
            list(chunk(tokens_without_stable, self.settings.price_batch_size)),
        )
    )
    return ([Price(token=stable, price=1)] if stable else []) + reduce(lambda l1, l2: l1 + l2, batches, [])


### Sign and send transaction

In [ ]:
#| export

@patch
@require_context
async def sign_and_send_tx(self: Chain, tx, wait: bool = True):
    spender = self.account.address
    tx = await tx.build_transaction({ 'from': spender, 'nonce': await self.web3.eth.get_transaction_count(spender) })
    signed_tx = self.account.sign_transaction(tx)
    tx_hash = await self.web3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return await self.web3.eth.wait_for_transaction_receipt(tx_hash) if wait else tx_hash

### Set and check token allowance

In [ ]:
#| export

@patch
@require_context
async def set_token_allowance(self: Chain, token: Token, addr: str, amount: int):
    ERC20_ABI = [{
        "name": "approve",
        "type": "function",
        "constant": False,
        "inputs": [{"name": "spender", "type": "address"}, {"name": "amount", "type": "uint256"}],
        "outputs": [{"name": "", "type": "bool"}]
    }]
    token_contract = self.web3.eth.contract(address=token.token_address, abi=ERC20_ABI)
    return await self.sign_and_send_tx(token_contract.functions.approve(addr, amount))

@patch
@require_context
async def check_token_allowance(self: Chain, token: Token, addr: str) -> int:
    ERC20_ABI = [{
        "name": "allowance",
        "type": "function",
        "constant": True,
        "inputs": [{"name": "owner", "type": "address"}, {"name": "spender", "type": "address"}],
        "outputs": [{"name": "", "type": "uint256"}]
    }]
    token_contract = self.web3.eth.contract(address=token.token_address, abi=ERC20_ABI)
    return await token_contract.functions.allowance(self.account.address, addr).call()


### Deposit

In [ ]:
#| export

@patch
@require_context
async def deposit(self: Chain, deposit: Deposit, delay_in_minutes: float = 30, slippage: float = 0.01):
    amount_token0, pool, router_contract_addr = deposit.amount_token0, deposit.pool, self.settings.router_contract_addr
    print(f"gonna deposit {amount_token0} {pool.token0.symbol} into {pool.symbol} from {self.account.address}")
    [token0_amount, token1_amount, _] = await self.router.functions.quoteAddLiquidity(
        pool.token0.token_address,
        pool.token1.token_address,
        pool.is_stable,
        pool.factory,
        float_to_uint256(amount_token0, pool.token0.decimals),
        MAX_UINT256
    ).call()
    print(f"Quote: {pool.token0.symbol} {token0_amount / 10 ** pool.token0.decimals} -> {pool.token1.symbol} {token1_amount / 10 ** pool.token1.decimals}")

    # set up allowance for both tokens
    print(f"setting up allowance for {pool.token0.symbol}")
    await self.set_token_allowance(pool.token0, router_contract_addr, token0_amount)

    print(f"setting up allowance for {pool.token1.symbol}")
    await self.set_token_allowance(pool.token1, router_contract_addr, token1_amount)

    # check allowances
    token0_allowance = await self.check_token_allowance(pool.token0, router_contract_addr)
    token1_allowance = await self.check_token_allowance(pool.token1, router_contract_addr)

    print(f"allowances: {token0_allowance}, {token1_allowance}")

    # adding liquidity

    params = [
        pool.token0.token_address,
        pool.token1.token_address,
        pool.is_stable,
        token0_amount,
        token1_amount,
        apply_slippage(token0_amount, slippage),
        apply_slippage(token1_amount, slippage),
        self.account.address,
        get_future_timestamp(delay_in_minutes)
    ]

    print(f"adding liquidity with params: {params}")

    return await self.sign_and_send_tx(self.router.functions.addLiquidity(*params))

## OP Chain

In [ ]:
#| export

class OPChain(Chain):
    def __init__(self, **kwargs): super().__init__(make_op_chain_settings())


## Base Chain

In [ ]:
#| export

class BaseChain(Chain):
    def __init__(self, **kwargs): super().__init__(make_base_chain_settings())

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()